In [ ]:
import openpyxl
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import numpy as np
import nltk
nltk.download('all')


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
pip install pandas requests beautifulsoup4


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 4.3 MB/s eta 0:00:00


In [ ]:
import csv

import xlsxwriter


In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import cmudict

In [ ]:
# CALCULATION OF STOP WORDS

a1 = np.loadtxt("/content/drive/MyDrive/StopWords_Auditor.txt",delimiter=',', skiprows=0, dtype=str)
a = open('/content/drive/MyDrive/StopWords_Currencies.txt', 'r', encoding="ISO-8859-1")
a2 = []
for i in a:
    i=i.split(" ")
    a2.append(i[:len(i)-1][0])
a3 = np.loadtxt("/content/drive/MyDrive/StopWords_DatesandNumbers.txt",delimiter=',', skiprows=0, dtype=str)
a4 = np.loadtxt("/content/drive/MyDrive/StopWords_Generic.txt",delimiter=',', skiprows=0, dtype=str)
a5 = np.loadtxt("/content/drive/MyDrive/StopWords_GenericLong.txt",delimiter=',', skiprows=0, dtype=str)
a6 = np.loadtxt("/content/drive/MyDrive/StopWords_Geographic.txt",delimiter=',', skiprows=0, dtype=str)
a7 = np.loadtxt("/content/drive/MyDrive/StopWords_Names.txt",delimiter=',', skiprows=0, dtype=str)
stop_words=[]
for i in a1:
    stop_words.append(i.lower())
for i in a2:
  stop_words.append(i.lower())
for i in a3:
    i=i.split(" ")
    stop_words.append(i[0].lower())
for i in a4:
    stop_words.append(i.lower())
for i in a5:
    stop_words.append(i.lower())
for i in a6:
    i=i.split(" ")
    stop_words.append(i[0].lower())
for i in a7:
    i=i.split(" ")
    stop_words.append(i[0].lower())
# NEGATIVE AND POSITIVE WORDS
negative_word = open('/content/drive/MyDrive/negative-words.txt', 'r', encoding="ISO-8859-1")
negative_words = {}
for i in negative_word:
  i=i.lower()
  i=i[:len(i)-1]
  if i not in stop_words:
    negative_words[i]=1

positive_word = open('/content/drive/MyDrive/positive-words.txt', 'r', encoding="ISO-8859-1")
positive_words = {}
for i in positive_word:
  i=i.lower()
  i=i[:len(i)-1]
  if i not in stop_words:
    positive_words[i]=1



path='/content/drive/MyDrive/Colab Notebooks/Input.xlsx'
df = pd.read_excel(path)
res=[]
title=[]
r=1
special = '~`!@#$%^&*()"“"”''_+’=-0987654321;:?><.,'
for index, row in df.iterrows():
    url = row['URL']
    url_id = row['URL_ID']

    response = requests.get(url)
    content = response.content
    soup = BeautifulSoup(content, 'html.parser')
    # extraction of article title
    article_title = soup.title.text
    data = ''
    article_texts=[]
    syllable_per_word = []
    number_of_personal_pronouns = 0
    average_word_length = []
    # extraction of article text only
    for data in soup.find_all("p",class_=""):
        article_texts.append(data.get_text())
    article_txt=[]
    st=0
    at=0
    cw=0

    for i in article_texts:

      word_tkn=word_tokenize(i)
      sent_tkn = sent_tokenize(i)
      # WORD SYLLABLES
      for word in word_tkn:
        average_word_length.append(len(word))
        if(word!='US'):
          word=word.lower()
        if (word == 'i' or word == 'we' or word =='my' or word =='ours' or word=='us'):
          number_of_personal_pronouns+=1
        su=0
        d = {}.fromkeys('aeiou',0)
        h = False
        for x in word:
            if x in d:
                d[x] += 1
        for q in d.values():
          su=su+q
        if word.endswith("es") or word.endswith("ed"):
          su-=1
        syllable_per_word.append(su)
        if su>2:
            cw += 1
      # TOTAL NUMBER OF SENTENCES
      for item in sent_tkn:
        st=st+1
      #  TOTAL NUMBER OF WORDS
      for j in word_tkn:
        if j not in special:
          article_txt.append(j)
          at=at+1

      for item in range(len(average_word_length)):
        average_word_length[item]=average_word_length[item]/at

    # FINAL ARTICLE TEXT AFTER CLEANING
    final_article_text=[]
    for i in article_txt:
      if i not in stop_words:
        final_article_text.append(i)

    # VARIABLES
    total_words = len(final_article_text)
    positive_score = 0
    negative_score = 0
    polarity_score = 0
    subjectivity_score = 0
    for i in final_article_text:
      if i in positive_words:
        positive_score+=1
      elif i in negative_words:
        negative_score-=1
    negative_score = negative_score *(-1)
    polarity_score =  (positive_score - negative_score)/ ((positive_score + negative_score) + 0.000001)
    subjectivity_score = (positive_score + negative_score)/ ((total_words) + 0.000001)


    if(st!=0):
      average_sentence_length = at/st
      average_number_of_words_per_sentence = at/st
    percentage_of_complex_words = cw + at
    fog_index = 0.4 * (average_sentence_length + percentage_of_complex_words)




    word_count = total_words

    complex_word_count = cw

    res.append([positive_score,negative_score,polarity_score,subjectivity_score,average_sentence_length,percentage_of_complex_words,fog_index,average_number_of_words_per_sentence,word_count,complex_word_count,syllable_per_word,number_of_personal_pronouns,average_word_length ])
    title.append(["URL_ID","URL","positive_score","negative_score","polarity_score","subjectivity_score","average_sentence_length","percentage_of_complex_words","fog_index","average_number_of_words_per_sentence","word_count","complex_word_count","syllable_per_word","number_of_personal_pronouns","average_word_length"])
    print(positive_score,negative_score,polarity_score,subjectivity_score,average_sentence_length,percentage_of_complex_words,fog_index,average_number_of_words_per_sentence,word_count,complex_word_count,syllable_per_word,number_of_personal_pronouns,average_word_length )
workbook = xlsxwriter.Workbook('/content/drive/MyDrive/Colab Notebooks/Output Data Structure.xlsx')
worksheet = workbook.add_worksheet()
r=0
c=2
for i in range(15):
  worksheet.write(r, i, title[0][i] )
r=r+1
ip=37

for index, row in df.iterrows():
    url = row['URL']
    url_id = row['URL_ID']
    worksheet.write(ip-36,1,url)
    worksheet.write(ip-36,0,url_id)
    ip+=1

for i in res:
  c=2
  for j in i:
    worksheet.write(r,c,str(j))
    c=c+1
  r=r+1
workbook.close()






60 32 0.30434782277882805 0.08906098732907938 22.0 2361 953.2 22.0 1033 579 [5, 0, 1, 2, 1, 2, 0, 3, 3, 1, 1, 1, 1, 2, 0, 1, 1, 1, 1, 1, 5, 2, 2, 1, 1, 1, 0, 1, 2, 1, 3, 0, 1, 1, 0, 0, 2, 1, 1, 0, 4, 1, 0, 0, 1, 2, 1, 1, 1, 3, 1, 3, 4, 0, 1, 1, 1, 0, 4, 0, 4, 2, 1, 1, 0, 1, 1, 1, 3, 1, 3, 4, 1, 1, 5, 5, 0, 1, 2, 2, 3, 3, 2, 1, 1, 4, 6, 1, 3, 1, 3, 1, 1, 2, 5, 1, 2, 2, 1, 3, 4, 0, 1, 1, 1, 1, 2, 0, 5, 5, 1, 2, 1, 1, 1, 3, 4, 1, 1, 4, 2, 0, 3, 2, 0, 5, 3, 1, 5, 3, 0, 2, 1, 1, 2, 1, 2, 1, 2, 1, 3, 0, 0, 0, 1, 1, 1, 2, 2, 3, 0, 3, 1, 2, 1, 1, 3, 1, 3, 0, 1, 1, 2, 1, 4, 2, 0, 0, 3, 1, 3, 2, 1, 1, 2, 2, 1, 1, 3, 2, 1, 4, 1, 2, 1, 3, 3, 1, 1, 2, 0, 1, 2, 1, 2, 2, 1, 2, 1, 1, 1, 3, 1, 2, 2, 2, 1, 2, 1, 1, 1, 5, 0, 2, 2, 3, 1, 1, 1, 2, 1, 2, 2, 3, 1, 2, 1, 5, 1, 0, 1, 1, 1, 2, 1, 1, 0, 1, 2, 0, 1, 4, 2, 1, 1, 1, 2, 3, 1, 3, 4, 3, 0, 1, 3, 3, 1, 1, 4, 2, 1, 1, 2, 2, 0, 2, 1, 6, 2, 0, 2, 0, 0, 2, 1, 1, 1, 4, 3, 1, 3, 3, 1, 2, 1, 3, 1, 1, 1, 1, 1, 3, 1, 3, 1, 0, 4, 1, 3, 1, 1, 1, 1, 2, 1, 1, 1, 2,